# **Predicción de Precios de Acciones de NVIDIA (NVDA) con Deep Learning**

### **Proyecto Final Samsung Innovation Campus – Módulo de Inteligencia Artificial (EC04)**

Integrantes:

- Ulices Chingo
- Sofia Feijóo
- Brayan Maisincho
- Esteban Quiña
- Alan Palma


In [ ]:
# Importar librerias de terceros
import numpy as np
import matplotlib.pyplot as plt

# Importar funciones propias
from src.model_LSTM import model_LSTM

2025-12-10 15:22:10.954601: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [ ]:
# Usar model_LSTM
window = 60
batch_size = 32
epochs = 50

# Llamar a la función model_LSTM con los parámetros definidos
model = model_LSTM(window_size=window, batch_size=batch_size, epochs=epochs, X_train=X_train, y_train=y_train)